In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision.datasets as dt
import torchvision.transforms as tr

import datetime

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv_2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc_1 = nn.Linear(64 * 14 * 14, 128)
        self.fc_2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))

        x = self.pool(x)

        x = x.view(-1, 64 * 14 * 14)

        x = F.relu(self.fc_1(x))
        return self.fc_2(x)

In [ ]:
model = CNN()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
transform = tr.Compose([tr.ToTensor(), tr.Normalize((0.1307,), (0.3081))])

In [ ]:
train_dataset = dt.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = dt.MNIST(root="./data", train=False, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:00<00:00, 38.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.05MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.64MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.26MB/s]


In [ ]:
test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=0.3081)
           )

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [ ]:
def train(model, loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} Loss: {loss.item():4f}")

In [ ]:
for epoch in range(1, 3):
    train(model, train_dataloader, optimizer, criterion, epoch)

Train Epoch: 1 Loss: 0.095846
Train Epoch: 1 Loss: 0.085923
Train Epoch: 1 Loss: 0.114507
Train Epoch: 1 Loss: 0.043729
Train Epoch: 1 Loss: 0.075596
Train Epoch: 1 Loss: 0.005940
Train Epoch: 1 Loss: 0.004005
Train Epoch: 1 Loss: 0.066660
Train Epoch: 1 Loss: 0.054196
Train Epoch: 1 Loss: 0.035267
Train Epoch: 2 Loss: 0.028869
Train Epoch: 2 Loss: 0.015838
Train Epoch: 2 Loss: 0.014496
Train Epoch: 2 Loss: 0.099183
Train Epoch: 2 Loss: 0.010527
Train Epoch: 2 Loss: 0.019976
Train Epoch: 2 Loss: 0.004302
Train Epoch: 2 Loss: 0.007665
Train Epoch: 2 Loss: 0.175157
Train Epoch: 2 Loss: 0.002625


In [ ]:
def test(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in loader:
            output = model(data)
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)

    acc = 100. * correct / total
    print(f"Test accuracy: {acc:.2f}%")

In [ ]:
test(model, test_dataloader)

Test accuracy: 99.01%
